In [329]:
import chipwhisperer as cw
import time

# Connect to the ChipWhisperer hardware
scope = cw.scope()
target = cw.target(scope)
target.baud = 9600

In [380]:
# GLITCH 
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "enable_only"
scope.glitch.output = "glitch_only"
scope.glitch.trigger_src = "manual"
scope.glitch.trigger_src = "ext_continuous"
scope.glitch.repeat = 22
scope.glitch.ext_offset = 1  
    
# CLOCK
# scope.clock.clkgen_freq = 32E6 # Overclocking can make glitching better, max freq: 81Mhz
scope.clock.clkgen_freq = 96E6

# IO
scope.io.glitch_lp = True
scope.io.glitch_hp = True
scope.io.tio1 = "high_z" # Read pin from Renesas to see if glitch worked
scope.io.tio2 = "serial_rx" # Can be used to receive serial 
scope.io.tio3 = "gpio_high" # To /RESET pin
scope.io.tio4 = "high_z"
scope.io.hs2 = "clkgen" # Attach to clockinput Renesas chip EXTAL
scope.io.hs2 = "glitch" # Attach to clockinput Renesas chip EXTAL

# TRIGGER
scope.trigger.triggers = "tio4"

print(scope.trigger)

def reset_mcu():
    """Reset the microcontroller by toggling IO3 low."""
    # print(".", end="")
    scope.io.tio3 = "gpio_low"  # Set IO3 as GPIO to reset the MCU
    time.sleep(0.01)  # Wait for 10ms
    scope.io.tio3 = "gpio_high"  # Set IO3 as GPIO to reset the MCU
    # time.sleep(0.01)  # Wait for 10ms
    target.flush()


reset_mcu()

# Good settings:
# 32Mhz / 16 repeat / 3.345V
# 50 tries
# successes = 18, repeat = 15
# successes = 10, repeat = 16
# successes = 20, repeat = 17



triggers = tio4
module   = basic



In [382]:
# scope.io.tio1 = "serial_tx"
# scope.io.tio2 = "serial_rx"
# target.baud = 9600


# # Send data (optional)
# target.write("Hello, ChipWhisperer!\n")
# print("start")

# # Receive data
while True:
    if target.in_waiting:
        #print("something available: ", target.in_waiting, target.read(0))
        data = target.read()
        if len(data) > 0:
            print(data)


#         time.sleep(1)

 
S
 
S
 
S


KeyboardInterrupt: 

In [378]:
# gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset", "tries"])
#gc = cw.GlitchController(groups=["success", "reset", "normal", "bigsuccess"], parameters=["ext_offset", "repeat", "tries"])
gc = cw.GlitchController(groups=["success", "reset", "normal", "bigsuccess"], parameters=["width", "offset", "repeat", "tries"])
gc.display_stats()


num_tries = 1 # increase to get better glitch stats
gc.set_range("tries", 1, num_tries)
# gc.set_range("ext_offset", 1000, 1000)
gc.set_range("repeat", 1, 1000)
gc.set_range("width", -48, 48)
gc.set_range("offset", 0, 48)
# gc.set_range("ext_offset", 7, 10)

gc.set_global_step(0.5)


gc.set_step("width", 0.5)
gc.set_step("offset", 1)
# gc.set_step("ext_offset", 1)
gc.set_step("repeat", 500)
# gc.set_step("tries", 1)

# scope.glitch.repeat = 11



IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

IntText(value=0, description='bigsuccess count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='width setting:', disabled=True, max=10.0, readout…

FloatSlider(value=0.0, continuous_update=False, description='offset setting:', disabled=True, max=10.0, readou…

FloatSlider(value=0.0, continuous_update=False, description='repeat setting:', disabled=True, max=10.0, readou…

FloatSlider(value=0.0, continuous_update=False, description='tries setting:', disabled=True, max=10.0, readout…

In [379]:
import matplotlib.pyplot as plt
#disable logging
cw.set_all_log_levels(cw.logging.CRITICAL)

reset_mcu()
scope.arm()

# scope.glitch.trigger_src = "manual"
loff = scope.glitch.offset
scope.trigger.edges = 'falling_edge'   
lwid = scope.glitch.width
total_successes = 0
successes = 0
resets = 0
highest_success_rate = -1
best_repeat_value = -1

scope.glitch.repeat = 1
scope.glitch.ext_offset = 1000

results = []  # To store the success counts for each repeat value
parameter_values = []

def glitch_worked():
    return not scope.io.tio_states[0]
    # if scope.io.tio_states[0]:
    #     return False
    # else:
    #     return True
    # for _ in range(10):
    #     if scope.io.tio_states[0]:
    #         return False
    #     # print(scope.io.tio_states[0])
    #     time.sleep(0.1)
        
    # return True


for glitch_setting in gc.glitch_values():
    scope.glitch.width = glitch_setting[0]
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.repeat = glitch_setting[2]
    try_counter = glitch_setting[3]

    reset_mcu()
    # print(scope.io.tio_states)
    # for _ in range(100):
    #     print(scope.io.tio_states)
    #     time.sleep(0.01)



    # scope.arm()
    time.sleep(0.1)  # Short delay for to check if it worked

    if glitch_worked():
        
        successes += 1
        print(f"SUCCES: glitch width: {glitch_setting[0]} offset: {glitch_setting[1]}")
        # time.sleep(1)
            # print(f"Testing glitch try: {glitch_setting[1]}")
        gc.add("success") 
            # print("start sleep")
        # for _ in range(2):
        #     print(scope.io.tio_states[0])
        #     time.sleep(1)
            # print("stop sleep")
    else:
        gc.add("normal")

    if try_counter == num_tries: # New repeat value has started

        if(successes > 0):
            rate  = successes / num_tries
            if rate > highest_success_rate:
                 highest_success_rate = rate
                 best_parameter_value = scope.glitch.width
            print("successes = {}, repeat = {}".format(successes / num_tries, scope.glitch.width))

        results.append(round(rate, 2))  # Append success count for each repeat value
        parameter_values.append(scope.glitch.ext_offset)
        successes = 0

    
    time.sleep(0.0)
    if target.in_waiting:
        #print("something available: ", target.in_waiting, target.read(0))
        data = target.read()
        if len(data) > 0:
            if "BF" not in data:
                if data.startswith("B"):
                    data = data[1:]

                if "B" in data:
                    gc.add("reset")
                
                if data.startswith("S") and len(data) == 5:
                    print("big")
                    gc.add("bigsuccess")
                print(data)


        # time.sleep(1)



print("Done glitching")


print("Best", rate, num_tries, best_parameter_value)

# plt.figure()
# plt.plot(parameter_values, results, marker='o')
# plt.title("Glitch Success Count vs Repeat Value")
# plt.xlabel("Repeat Value")
# plt.ylabel(f"Success Count {num_tries}")
# plt.grid()
# plt.show()


#enable logging
cw.set_all_log_levels(cw.logging.WARNING)
gc.plot_2d(plotdots={"success":"+g", "bigsuccess":"+b", "reset":"xr", "normal":None}, x_index="width", y_index="offset")

ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
SUCCES: glitch width: -48 offset: 30
successes = 1.0, repeat = -48.046875
S265
ÿ
SUCCES: glitch width: -48 offset: 32
successes = 1.0, repeat = -48.046875
S265
SUCCES: glitch width: -48 offset: 33
successes = 1.0, repeat = -48.046875
S265
ÿ
SUCCES: glitch width: -48 offset: 35
successes = 1.0, repeat = -48.046875
S
ÿ
ÿ
SUCCES: glitch width: -48 offset: 38
successes = 1.0, repeat = -48.046875
S265
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
SUCCES: glitch width: -48 offset: 45
successes = 1.0, repeat = -48.046875
S265
SUCCES: glitch width: -48 offset: 46
successes = 1.0, repeat = -48.046875
big
S2465
ÿ
ÿ
SUCCES: glitch width: -47.5 offset: 0
successes = 1.0, repeat = -47.65625
S265
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
ÿ
SUCCES: glitch width: -47.5 offset: 29
successes = 1.0, repeat = -47.65625
S265
ÿ
ÿ
SUCCES: glitch width: -47.5 offset: 32
successes = 1.0, repeat = -47.65625
S265
ÿ
ÿ
ÿ
ÿ
ÿ
SUCCES: glitch width: -47.5 offset: 38
successes = 1

:Overlay
   .Points.I          :Points   [x,y]
   .Points.Success    :Points   [width,offset]   (success_rate)
   .Points.Reset      :Points   [width,offset]   (reset_rate)
   .Points.Bigsuccess :Points   [width,offset]   (bigsuccess_rate)

In [352]:
# gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None})
gc.plot_2d(plotdots={"success":"+g", "reset":"xr", "normal":None}, x_index="ext_offset", y_index="repeat")
# results = gc.calc(sort="success_rate")
# results
# gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None}, )
# print(gc.results)

:Overlay
   .Points.I       :Points   [x,y]
   .Points.Success :Points   [ext_offset,repeat]   (success_rate)
   .Points.Reset   :Points   [ext_offset,repeat]   (reset_rate)

In [190]:
scope.glitch.repeat = 100
scope.glitch.width = 25
scope.clock.clkgen_freq = 96E6
scope.glitch.output = "enable_only"
# scope.glitch.trigger_src = "manual"

for _ in range(1000):
    scope.arm()
    # reset_mcu()
# gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None})
# gc.plot_2d(plotdots={"success":"+g", "reset":"xr", "normal":None}, x_index="repeat", y_index="tries")
# results = gc.calc(ignore_params=["ext_offset"], sort="success_rate")
# results
# gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None}, )
# print(gc.)

KeyboardInterrupt: 

In [ ]:
scope.dis()
target.dis()